# Image Recommendation System via Collaborative Filtering

# ***Please read the instructions very carefully***

1.   Assignment must be implemented in Python 3 only.
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for evaluation metrics, data visualization (matplotlib etc.).
3.   You will be evaluated not just on the overall performance of the model and also on the experimentation with hyper parameters, data prepossessing techniques etc.
4.   ⚠️ The Assignment will be evaluated automatically. Please adhere to taking proper inputs from `config.csv` file. You can change your `config.csv` file to experiment with your code. But at the end, make sure that your outputs are corresponding to input values in `config.csv`
5.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

## About the Dataset
Behance is a community art website where users showcase and discover creative work. Each user is able to “appreciate” (equivalent to a “like” on Instagram or a “react” on Facebook) an image, indicating that they like the image. It is in the website’s best interests to show users pictures that they would like, to keep them engaged for longer. For this question, given a set of pictures that a user has already appreciated, you have to show them a new picture that they would like based on what similar users appreciated.


<br><br>
**The dataset has information of 1 million appreciates of 63,497 users on 178,788 items. The file Behance appreciate 1M has a triplet in each line in the form of (user id, item id, unix timestamp).**

**Task: Take the inputs from the config.csv file and output the recommendations for a particular person**
- Collaborative Filtering is a way to predict items to the user based on the the
user’s history and the history of similar users. The similarity between users can be quantified by the number of images that both the users appreciated.
- The images appreciated by a similar user would be the most suitable images to show a user. Since we can find the similarity between any two users, we would be able to find the “nearest” neighbours of any user, allowing us to use a KNN-based algorithm to recommend new images to a user.
- Since people do not like seeing pictures that they have seen already. Make sure that you do not recommend pictures that a user has appreciated already.
- Output the final response will be saved in the file named ```config['output_file']```.


**Output file format:**
Populate the output file with images that the user has not seen of the k most
similar users, in descending order of their similarity. Each line in the output
file should be a duplet in the form of (item id, user id), where the user id is the
id of the kth similar user. The order of the images corresponding to the same
similar user would not matter. The output file would look something like this:
```
item_id_1_of_1st_similar_user 1st_most_similar_user_id
item_id_2_of_1st_similar_user 1st_most_similar_user_id
item_id_3_of_1st_similar_user 1st_most_similar_user_id
...
item_id_1_of_2nd_similar_user 2nd_most_similar_user_id
item_id_2_of_2nd_similar_user 2nd_most_similar_user_id
item_id_3_of_2nd_similar_user 2nd_most_similar_user_id
...
item_id_1_of_kth_similar_user kth_most_similar_user_id
item_id_2_of_kth_similar_user kth_most_similar_user_id
item_id_3_of_kth_similar_user kth_most_similar_user_id
```

The dataset was extracted using Behance’s API as a part of the paper
“Vista: A visually, socially, and temporally-aware model for artistic
recommendation, RecSys, 2016”. Check out this [Google Drive folder](https://drive.google.com/drive/folders/0B9Ck8jw-TZUEc3NlMjVXdDlPU1k?resourcekey=0-6_8ykn0o4fLc5fuTEm91xA) for
more information about the dataset.


Have fun! The users are waiting to see new pictures!

# CODE

### Import necessary libraries

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Config Generation Sample Code.
# ⚠️ Only for experimentation on your side.
# ⚠️ Should be commented during submission.

# df = pd.DataFrame(data=[{'id':276633,
#                   'k':5,
#                   'dataset_file':'./Behance_appreciate_1M',
#                   'output_file':'./output.txt'}])
# df.to_csv('config.csv')

In [3]:
config = pd.read_csv('config.csv').iloc[1]

In [4]:
config

Unnamed: 0                              0
id                                 276633
k                                       5
dataset_file      ./Behance_appreciate_1M
output_file     ./testing/output_test.txt
Name: 1, dtype: object

In [5]:
user = int(config['id'])
k_value = int(config['k'])

### Read the Data

In [6]:
with open(config['dataset_file'], 'r') as inFile:
    appreciate_data = inFile.readlines()

### Initialize a dictionary to store the item_ids that a user likes

### Go through each line of the input file and construct the user_likes dictionary

In [7]:
from scipy.sparse import csr_matrix,lil_matrix
data=lil_matrix((3957261,2497166),dtype=bool)

In [8]:
for line in appreciate_data:
    line = line.strip()
    user_id = int(line.split()[0])
    item_id = int(line.split()[1])
    data[user_id,item_id]=True

### Use KNN after Collaborative Filtering to find nearest neighbors

In [9]:
def intersection(list1, list2):
    """ returns intersection of two lists """
    val=0
    for i in range(len(list1)):
        if list1[i] and list2[i] : 
            val+=1
    return val

def unoin(list1, list2):
    """ returns length of union of two lists """
    val=0
    for i in range(len(list1)):
        if list1[i] or list2[i] : 
            val+=1
    return val

def jacard_similarity_score(list1, list2):
    """ returns jacard similarity score of two lists """
    return intersection(list1, list2) / unoin(list1, list2)

def cosine_similarity_score(list1, list2):
    """ returns cosine similarity score of two lists """
    return intersection(list1, list2) / (np.sqrt(len(list1)*len(list2)))

In [10]:
jacard_similarity_score(np.array(data[276633].todense())[0],np.array(data[4789].todense())[0])

0.0

In [11]:
def neighbors(user,k_value,user_likes):
    """ returns an iterable object (like list or generator) """
    scores = {}
    test=np.array(data[user].todense())
    for x in range(user_likes.shape[0]):
        if x != user:
            scores[x] = jacard_similarity_score(test[0],np.array(data[x].todense())[0])
    return sorted(scores.items(), key=lambda x:x[1], reverse=True)[:k_value]


In [22]:
data[0]

<1x2497166 sparse matrix of type '<class 'numpy.bool_'>'
	with 0 stored elements in List of Lists format>

In [17]:
from sklearn.decomposition import TruncatedSVD
tt=TruncatedSVD(n_components=2)
X=tt.fit_transform(data)

In [18]:
X=np.array(X)

In [30]:
cc=TruncatedSVD(n_components=2)
Y=cc.fit_transform(data[user])

/home/archit/.local/lib/python3.8/site-packages/sklearn/decomposition/_truncated_svd.py:205: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var


In [32]:
Y

array([[14.24780685]])

In [25]:
import scann
searcher = scann.scann_ops_pybind.builder(X, 10, "dot_product").tree(
    num_leaves=2000, num_leaves_to_search=100, training_sample_size=250000).score_ah(
    2, anisotropic_quantization_threshold=0.2).reorder(100).build()

2022-01-23 21:14:14.648380: I scann/partitioning/partitioner_factory_base.cc:71] Size of sampled dataset for training partition: 249712
2022-01-23 21:14:15.862130: I ./scann/partitioning/kmeans_tree_partitioner_utils.h:102] PartitionerFactory ran in 1.213016294s.


In [34]:
neighbors, distances = searcher.search_batched(Y)

RuntimeError: Error during search: Query doesn't match dataset dimsensionality

In [ ]:
scores=neighbors(user,k_value,data)

KeyboardInterrupt: 

### Open the output file to write all the lines to the file

In [ ]:
outFile = open(config['output_file'], 'w')

for n_user in neighbors(user,k_value,user_likes):
    user_id = n_user[0]
    for item_id in diff_list(user_likes[user_id],user_likes[user]):
        outFile.write(str(item_id) + ' ' + str(user_id) + '\n')

outFile.close()